In [ ]:
# this notebook compiles all the downloaded census data into a set of cleaned files. 

# specifically, it outputs a county-level and school-district csv file with the columns:
# geo_id (fips or SD code) | geo_name | geo_type | state | x_0 | x_1 | ... | x_{n-1} ,
# where x_i is the most recently available variable of interest for that geo_id


import pandas as pd
from typing import List, Dict
from functools import reduce

import numpy as np
import warnings
warnings.filterwarnings("ignore",category=pd.errors.DtypeWarning)

In [ ]:
# for each data dir:
# have county and school district data
# want a specific column or list of columns, for instance:
# "S1602_C04_001E","Estimate!!Percent limited English-speaking households!!All households"

# from itertools import product

NULL_CHARS = ["-","N"]

def retrieve_variables(varname: str, code: str, cols_and_names: Dict[str,str]) -> pd.DataFrame:
    """
    Construct a pandas dataframe containing the columns specified in `cols_and_names`, by accessing downloaded ACS directories and retrieving 
    most recently available variables of interest. `varname` and `code` correspond to the directory and file names of the downloaded data.

    Returns a dataframe with the `geo_id` of the county or school district, as well as a source column for each specified colname 
    containing the survey year that the data was pulled from.
    """
    cols_to_rename = {"GEO_ID":"geo_id"} | cols_and_names
    # colnames = list ( cols_to_rename.values() ) 
    

    def load_df(path) -> pd.DataFrame:
        # helper method to retrieve the frame at `path`, rename it, and set up nulls
        df_ = pd.read_csv(path, skiprows = [1])[list(cols_to_rename)]
        df_ = df_.replace(NULL_CHARS, np.nan)
        df_ = df_.rename(columns=cols_to_rename)
        return df_

    fname = "../data/{varname}-{geo_level}/ACSST{version}.S{code}-Data.csv"

    geo_levels = ["cty","sd"] # county, school district
    versions   = ["5Y2021","1Y2021","1Y2022"] # this has to be sorted in ascending order of recency


    dfs = []
    for geo_level in geo_levels:

        vs = versions[0]
        df = load_df(fname.format(varname=varname,geo_level = geo_level, version=vs, code=code))

        for vs in versions[1:]: # logic to pull in newer demographic info when available
            # break

            df_newer = load_df(fname.format(varname=varname, geo_level = geo_level, version=vs, code=code))
            for c in cols_and_names.values(): 
                if c not in ["geo_id","geo_name"]:
                    
                    # match up the two dataframes by geo_id. the initial df has the most complete record
                    # so we should use its indices (left join)
                    temp = pd.merge(df, df_newer[['geo_id',c]], how="left", on="geo_id", suffixes=(None, "_recent"))

                    # add in any updated values present in thes newer data
                    temp.loc[~temp[c +"_recent"].isna(), c] = temp[c +"_recent"] 

                    # replace this in the df and delete temp.                
                    df[c] = temp[c]
                    del temp

        dfs.append(df)
        

    return pd.concat(dfs).reset_index(drop=True)
    

frames = [

    retrieve_variables("limited-english", "1602", {"NAME": "geo_name", "S1602_C04_001E": "pct_limited_english_households"}),
    retrieve_variables("poverty", "1701", {"S1701_C03_001E": "pct_pop_below_poverty_level"}),
    retrieve_variables("lang-at-home", "1601", {"S1601_C01_001E":"total_population_5+", "S1601_C02_002E": "pct_speaks_only_english", "S1601_C02_003E": "pct_speaks_other_than_english"}),
    retrieve_variables("snap","2201", {
            "S2201_C01_001E": "total_households",
            "S2201_C02_009E": "pct_households_with_chilren", 
            "S2201_C04_001E": "pct_households_receiving_food_stamps",
            "S2201_C02_010E": "pct_households_with_children_married_family",
        }
    ),
    retrieve_variables("ed-enrollment","1401", {"S1401_C06_001E": "pct_pop_3+_in_private_school", "S1401_C04_001E": "pct_pop_3+_in_public_school" }),
    retrieve_variables("income","1903",{"S1903_C03_016E":"median_income_households_with_children"}) # this variable has ann occasional value of 250_000+ which cannot be converted to a float
]

# inspiration for the extracted variablels are drawn from conversations with Jesse and a NCES dashboard (see [here](https://nces.ed.gov/Programs/Edge/ACSDashboard/0600014)) 
# 1.  population number (5y+)
# 2.  total household number
# 3.  percent of limited english households
# 4.  percent of population speaking english only and (percent of population speaking aother language than english) 
# 5.  percent of population below poverty level
# 6.  percent of households with children
# 7.  percent of households recieving food stamps 
# 8.  percent of households with children that are a married family
# 9.  percent of kids in school that are enrolled in a private school 
# 10. percent of kids in school that are enrolled in a public school
# 11. median income of households with children. (note: for a small number of high and low median incomes this variable will map to `250,000+` and `2,500-`. I set these values to be 250000 and 2500 respectively)



In [ ]:
def merge_on_geo(a: pd.DataFrame, b: pd.DataFrame) -> pd.DataFrame:
    return pd.merge(a,b,how='outer',on='geo_id')

df_all_vars = reduce(merge_on_geo, frames)

In [ ]:
numrical_cols = list(set(df_all_vars.columns) - {"geo_id","geo_name"})

df_all_vars["median_income_households_with_children"] = df_all_vars["median_income_households_with_children"].apply(lambda s: s.replace("+","").replace(",","").replace("-","") if isinstance(s,str) else s)
df_all_vars[numrical_cols] = df_all_vars[numrical_cols].astype(float)


df_all_vars["geo_type"] = df_all_vars["geo_id"].apply(lambda id: "CTY" if len(id) == 14 else "SD") # county codes are shorter

In [ ]:
df_all_vars.to_csv("cty-sd-latest-indicators.csv",index=False)